In [2]:
import os

In [3]:
%pwd

'c:\\Users\\koush\\OneDrive\\Desktop\\mouna\\Text-Summarizer-Project\\research'

In [4]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\koush\\OneDrive\\Desktop\\mouna\\Text-Summarizer-Project'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformerConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [13]:
import sys
import os

# Add src to sys.path
sys.path.append(os.path.join(os.getcwd(), "src"))

from textsummarizer.config_constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformerConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config =DataTransformerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            tokenizer_name= config.tokenizer_name
        )
        return data_transformation_config

In [10]:
!pip install transformers

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 10.0/10.0 MB 62.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 45.1 MB/s eta 0:00:00


In [15]:
import os
from textsummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [9]:

tok = AutoTokenizer.from_pretrained("t5-small")
print(type(tok))


<class 'transformers.models.t5.tokenization_t5_fast.T5TokenizerFast'>


In [16]:
class DataTransformation:
    def __init__(self, config: DataTransformerConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [15]:
!pip install protobuf==3.20.*

   ---------------------------------------- 0.0/904.4 kB ? eta -:--:--
   --------------------------------------- 904.4/904.4 kB 13.7 MB/s eta 0:00:00


In [16]:
!pip install tiktoken

   ---------------------------------------- 0.0/798.9 kB ? eta -:--:--
   --------------------------------------- 798.9/798.9 kB 16.8 MB/s eta 0:00:00


In [38]:
!pip install sentencepiece

In [17]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-09-08 17:28:48,452: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-08 17:28:48,455: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-08 17:28:48,456: INFO: common: created directory at: artifacts]
[2025-09-08 17:28:48,459: INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\koush\anaconda3\envs\textsummarisation\lib\site-packages\transformers\tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 116893.01 examples/s]
